In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
import gc

In [2]:
feature_list = ['lr','xgb','svc','rf','knn','mlp']
res_train = pd.read_csv('train_2s.csv')
res_test = pd.read_csv('test_2s.csv')
y_train = np.asarray(res_train.pop('label'))
x_train = np.asarray(res_train.drop(columns='svc'))
x_test = np.asarray(res_test.drop(columns='svc'))
print (np.shape(x_train), np.shape(y_train), np.shape(x_test))

(254386, 5) (254386,) (172956, 5)


In [3]:
kf = KFold(n_splits=8)
n_fold = 0
score = 0
test_pred_list = []

for train_index, valid_index in kf.split(x_train):
    n_fold += 1
    print ("========== Fold %d: =========="%n_fold)
    # split samples
    x_tra, x_val = x_train[train_index], x_train[valid_index]
    y_tra, y_val = y_train[train_index], y_train[valid_index]

    # train the model
    model = RandomForestClassifier(max_depth=6, n_jobs=-1)
    model.fit(x_tra, y_tra)
    
    # predict
    val_pred = model.predict_proba(x_val)[:,1]
    test_pred = model.predict_proba(x_test)[:,1]
    test_pred_list.append(test_pred)
    
    score += log_loss(y_val, val_pred)
    del x_tra, x_val, y_tra, model, val_pred, test_pred
    gc.collect()
    
# make the feature
test_pred_list = np.asarray(test_pred_list)
test_pred_list = np.mean(test_pred_list, axis=0)
test_pred_list = np.squeeze(test_pred_list)

print (score/8)
result = pd.DataFrame({'y_pre': test_pred_list})
result.to_csv('s_submission.csv', index=False)

========== Fold 1: ==========
========== Fold 2: ==========
========== Fold 3: ==========
========== Fold 4: ==========
========== Fold 5: ==========
========== Fold 6: ==========
========== Fold 7: ==========
========== Fold 8: ==========
0.16716086975268896


In [20]:
kf = KFold(n_splits=8)
n_fold = 0
score = 0
test_pred_list = []

for train_index, valid_index in kf.split(x_train):
    n_fold += 1
    print ("========== Fold %d: =========="%n_fold)
    # split samples
    x_tra, x_val = x_train[train_index], x_train[valid_index]
    y_tra, y_val = y_train[train_index], y_train[valid_index]

    # train the model
    model = xgb.XGBClassifier(
        #learning_rate = 0.02,
        n_estimators= 2000,
        max_depth= 6,
        min_child_weight= 3,
        gamma=0.9,                        
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        n_jobs= -1,
        scale_pos_weight=1)
    model.fit(x_tra, y_tra)
    
    # predict
    val_pred = model.predict_proba(x_val)[:,1]
    test_pred = model.predict_proba(x_test)[:,1]
    test_pred_list.append(test_pred)
    
    score += log_loss(y_val, val_pred)
    del x_tra, x_val, y_tra, model, val_pred, test_pred
    gc.collect()
    
# make the feature
test_pred_list = np.asarray(test_pred_list)
test_pred_list = np.mean(test_pred_list, axis=0)
test_pred_list = np.squeeze(test_pred_list)

print (score/8)
result = pd.DataFrame({'y_pre': test_pred_list})
result.to_csv('s_submission.csv', index=False)

========== Fold 1: ==========
========== Fold 2: ==========
========== Fold 3: ==========
========== Fold 4: ==========
========== Fold 5: ==========
========== Fold 6: ==========
========== Fold 7: ==========
========== Fold 8: ==========
0.17533636452844822
